# 第3回講義 宿題

## 課題
今Lessonで学んだことを元に、MNISTのファッション版 (Fashion MNIST、クラス数10) をソフトマックス回帰によって分類してみましょう。

Fashion MNISTの詳細については以下のリンクを参考にしてください。

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap03`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap03 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します。
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します。これを最終的な評価とします。

### データの読み込み (このセルは修正しないでください)

In [15]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('../data/x_train.npy') #Jupyterでやるときはファイルの場所を書き換える
    y_train = np.load('../data/y_train.npy')
    
    # テストデータ
    x_test = np.load('../data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

### ソフトマックス回帰の実装

In [26]:
%%writefile submission_code.py

x_train, y_train, x_test = load_fashionmnist()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#from sklearn.utils import shuffle

def softmax(x):
    # WRITE ME
    x -= x.max(axis=1, keepdims=True) # expのunderflow & overflowを防ぐ
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

# logの中身が0になるのを防ぐ
def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=x))

# weights
# W =  # WRITE ME
np.random.seed(62)#1-100で62が最大
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')
# b =  # WRITE ME
b = np.zeros(shape=(10,)).astype('float32')

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, t, eps=0.5): #eps=1からいじった
    # WRITE ME
    global W, b
    
    batch_size = x.shape[0]
    
    # 順伝播
    y = softmax(np.matmul(x, W) + b) # shape: (batch_size, 出力の次元数)
    
    # 逆伝播
    cost = (- t * np_log(y)).sum(axis=1).mean()
    delta = y - t # shape: (batch_size, 出力の次元数)
    
    # パラメータの更新
    dW = np.matmul(x.T, delta) / batch_size # shape: (入力の次元数, 出力の次元数)
    db = np.matmul(np.ones(shape=(batch_size,)), delta) / batch_size # shape: (出力の次元数,)
    W -= eps * dW
    b -= eps * db

    return cost

def valid(x, t):
    # WRITE ME
    y = softmax(np.matmul(x, W) + b)
    cost = (- t * np_log(y)).sum(axis=1).mean()
    
    return cost, y

for epoch in range(1):
    # オンライン学習
    # WRITE ME
    #x_train, y_train = shuffle(x_train, y_train)
    for x, t in zip(x_train, y_train):
        cost = train(x[None, :],t[None, :])
    cost, y_pred = valid(x_valid, y_valid)
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        epoch + 1,
        cost,
        accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    ))

y_test = softmax(np.matmul(x_test, W) + b)
y_test = y_test.argmax(axis=1)

submission = pd.Series(y_test, name='label')
submission.to_csv('submission_pred.csv', header=True, index_label='id')

Overwriting submission_code.py


In [24]:
print(y_pred.argmax(axis=1))
print(y_valid.argmax(axis=1))

[6 9 3 ... 6 6 1]
[6 9 3 ... 2 6 1]


In [25]:
print(y_test)

[0 6 9 ... 0 3 1]
